In [2]:
library(tidyverse)
library(repr)
library(tidymodels)
library(rvest)
library(stringr)
library(DBI)
library(dbplyr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

# Prediction of leaf species


## Introduction

### Dataset

The present dataset describes the physical attributes of 40 different leaf species ranging from simple to complex leaves. Simple leaves are considered as having a single leaf while complex leaves subdivide into multiple leaflets from one stem. This project aims to use classification to determine whether shape features, texture features, or a combination of both are most accurate at predicting leaf species. 

More specifically, the predictors in question are solidity, aspect ratio, uniformity, and average contrast. In this context, solidity and aspect ratio describe leaf shape with solidity measuring how well a given leaf fits within a convex shape and aspect ratio referring to the ratio of the distance between the furthest perpendicular points on a leaf. Further, uniformity and average contrast describe leaf texture based on “statistical properties of the intensity histograms of grayscale transformations of the original RGB images” (Silva et al., 2013). Uniformity measures the similarity in intensity values and average contrast gives the standard deviation of intensity. 

Silva, P. F., Marçal, A. R., & da Silva, R. M. (2013). Evaluation of features for leaf discrimination. Springer Lecture Notes in Computer Science, 7950, 197–204. https://doi.org/10.1007/978-3-642-39094-4_23 


### Our research question

Are shape features, texture features, or a combination of both more accurate at predicting leaf species?

Shape features: Solidity and Aspect ratio

Texture features: Uniformity and Average contrast


# Preliminary exploratory data analysis:


In [3]:
#read dataset
leaf<-read_csv("leaf.csv") 

colnames(leaf) <- c(X1 = "class", X2 = "specimen", X3 = "eccentricity", X4 = "aspect_ratio", 
                         X5 = "elongation", X6 = "solidity", X7 = "sto_convexity", X8 = "iso_factor", 
                         X9 = "max_indent_depth", X10 = "lobedness", X11 = "ave_intensity", X12 = "ave_contrast", 
                         X13 = "smoothness", X14 = "3rd_moment", X15 =  "uniformity", X16 = "entropy")
leaf_data<-mutate(leaf,class=as_factor(class))
leaf_data



Warning message:
“Duplicated column names deduplicated: '1' => '1_1' [2], '1' => '1_2' [7]”
Parsed with column specification:
cols(
  `1` = col_double(),
  `1_1` = col_double(),
  `0.72694` = col_double(),
  `1.4742` = col_double(),
  `0.32396` = col_double(),
  `0.98535` = col_double(),
  `1_2` = col_double(),
  `0.83592` = col_double(),
  `0.0046566` = col_double(),
  `0.0039465` = col_double(),
  `0.04779` = col_double(),
  `0.12795` = col_double(),
  `0.016108` = col_double(),
  `0.0052323` = col_double(),
  `0.00027477` = col_double(),
  `1.1756` = col_double()
)



class,specimen,eccentricity,aspect_ratio,elongation,solidity,sto_convexity,iso_factor,max_indent_depth,lobedness,ave_intensity,ave_contrast,smoothness,3rd_moment,uniformity,entropy
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2,0.74173,1.5257,0.36116,0.98152,0.99825,0.79867,0.0052423,0.0050016,0.0241600,0.090476,0.0081195,0.00270800,7.4846e-05,0.69659
1,3,0.76722,1.5725,0.38998,0.97755,1.00000,0.80812,0.0074573,0.0101210,0.0118970,0.057445,0.0032891,0.00092068,3.7886e-05,0.44348
1,4,0.73797,1.4597,0.35376,0.97566,1.00000,0.81697,0.0068768,0.0086068,0.0159500,0.065491,0.0042707,0.00115440,6.6272e-05,0.58785
1,5,0.82301,1.7707,0.44462,0.97698,1.00000,0.75493,0.0074280,0.0100420,0.0079379,0.045339,0.0020514,0.00055986,2.3504e-05,0.34214
1,6,0.72997,1.4892,0.34284,0.98755,1.00000,0.84482,0.0049451,0.0044506,0.0104870,0.058528,0.0034138,0.00112480,2.4798e-05,0.34068
1,7,0.82063,1.7529,0.44458,0.97964,0.99649,0.76770,0.0059279,0.0063954,0.0183750,0.080587,0.0064523,0.00227130,4.1495e-05,0.53904
1,8,0.77982,1.6215,0.39222,0.98512,0.99825,0.80816,0.0050987,0.0047314,0.0248750,0.089686,0.0079794,0.00246640,1.4676e-04,0.66975
1,9,0.83089,1.8199,0.45693,0.98240,1.00000,0.77106,0.0060055,0.0065640,0.0072447,0.040616,0.0016469,0.00038812,3.2863e-05,0.33696
1,10,0.90631,2.3906,0.58336,0.97683,0.99825,0.66419,0.0084019,0.0128480,0.0070096,0.042347,0.0017901,0.00045889,2.8251e-05,0.28082


In [4]:
#Select the column we need

leaf_select<- leaf_data %>%
            select(class,specimen,solidity,aspect_ratio,ave_contrast,uniformity)
leaf_select

class,specimen,solidity,aspect_ratio,ave_contrast,uniformity
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2,0.98152,1.5257,0.090476,7.4846e-05
1,3,0.97755,1.5725,0.057445,3.7886e-05
1,4,0.97566,1.4597,0.065491,6.6272e-05
1,5,0.97698,1.7707,0.045339,2.3504e-05
1,6,0.98755,1.4892,0.058528,2.4798e-05
1,7,0.97964,1.7529,0.080587,4.1495e-05
1,8,0.98512,1.6215,0.089686,1.4676e-04
1,9,0.98240,1.8199,0.040616,3.2863e-05
1,10,0.97683,2.3906,0.042347,2.8251e-05


In [5]:
##Spliting the data into "Training Data" and "Testing Data"
set.seed(100)
 
leaf_split <- initial_split(leaf_data, prop = 0.75, strata = "class")
leaf_train <- training(leaf_split)
leaf_test <- testing(leaf_split)


Warning message:
“Too little data to stratify. Unstratified resampling will be used.”


In [6]:
 
##Summarizing the Mean of the selected predictors that we are going to use in our prediction to compare the accuracies
 
leaf_train_mean <- leaf_train %>%
        group_by(class)%>%
        summarize(mean_solidity = mean(solidity), mean_aspect_ratio = mean(aspect_ratio), 
                  mean_uniformity = mean(uniformity), mean_ave_contrast = mean(ave_contrast))
leaf_train_mean



`summarise()` ungrouping output (override with `.groups` argument)



class,mean_solidity,mean_aspect_ratio,mean_uniformity,mean_ave_contrast
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.9806578,1.688511,5.297778e-05,0.05887033
2,0.9834875,2.251863,2.580389e-04,0.08884263
3,0.9206125,1.130450,1.559151e-04,0.14650288
4,0.9605643,1.278814,4.249357e-04,0.09762486
5,0.8336025,1.892075,2.697888e-04,0.13553750
6,0.7431186,1.155943,6.147314e-05,0.07042200
7,0.8834478,1.800556,1.501821e-04,0.10520300
8,0.9658790,6.206540,1.644665e-04,0.10001430
9,0.9042470,1.275700,6.766000e-04,0.15695100


# Method

In [ ]:
KNN regression 

# Expected Outcomes and Significance